In [47]:
import scipy.stats
class VotingScheme:
    def __init__(self, preferences, voting_vector):
        self.preferences = preferences
        self.voting_vector = voting_vector # must be the length of possible outcomes

    def execute_voting(self, preferences):
        outcome_scores = {}

        # Initialize outcomes with zero scores
        for voter_prefs in preferences.values():
            for outcome in voter_prefs:
                outcome_scores[outcome] = 0

        # Calculate scores based on preferences and voting vector
        for voter, prefs in preferences.items():
            for i, outcome in enumerate(prefs):
                outcome_scores[outcome] += self.voting_vector[i]

        # Sort outcomes by score and then alphabetically to break ties
        outcomes = sorted(outcome_scores.items(), key=lambda x: (-x[1], x[0]))
        outcome = [outcome for outcome, score in outcomes]

        return outcome, outcome_scores


    def calc_overall_happiness(self, outcome):
        happiness_score = 0.0
        num_voters = len(self.preferences)

        for voter, prefs in self.preferences.items():
            happiness_score += self.calc_ind_happiness(voter,outcome)

        return happiness_score / num_voters

    def calc_ind_happiness(self, voter, outcome_ranking):

      spearman_correlation, _ = scipy.stats.spearmanr(self.preferences[voter], outcome_ranking)
      return spearman_correlation

In [48]:
preferences = {'A': ['o_1', 'o_3', 'o_2','o_4'], 'B': ['o_2', 'o_1', 'o_3','o_4'], 'C': ['o_2', 'o_3', 'o_1','o_4'], 'D': ['o_1','o_2','o_3','o_4'], 'E': ['o_4','o_3','o_2','o_1'],'F': ['o_4','o_3','o_2','o_1'],'G': ['o_3','o_2','o_1','o_4']}

# Priority Voting
priority_scheme = VotingScheme(preferences, [1, 0, 0, 0])
outcome, results = priority_scheme.execute_voting(preferences)
print(outcome)
print(results)
print(priority_scheme.calc_overall_happiness(outcome))
# Borda Count (Assuming 4 outcomes)
borda_scheme = VotingScheme(preferences, [3, 2, 1, 0])
outcome, results = borda_scheme.execute_voting(preferences)
print(outcome)
print(results)
print(borda_scheme.calc_overall_happiness(outcome))


['o_1', 'o_2', 'o_4', 'o_3']
{'o_1': 2, 'o_3': 1, 'o_2': 2, 'o_4': 2}
-0.057142857142857106
['o_2', 'o_3', 'o_1', 'o_4']
{'o_1': 10, 'o_3': 13, 'o_2': 13, 'o_4': 6}
0.34285714285714286


In [55]:
import itertools
import pandas as pd
import random

class MultiAgentTVAModel:
    """
    Represents a multi-agent Tactical Voting Analyst model.
    """

    def __init__(self, preferences, voting_scheme):
        """
        Initializes the model.

        Args:
            preferences: A dictionary representing the true preferences of each agent.
                         Format example: { agent_id: [preference_1, preference_2, ...] }
            voting_scheme:  An object representing the voting scheme in use.
        """

        self.preferences = preferences
        self.voting_scheme = voting_scheme
        self.agents = list(preferences.keys())  # Extract agent IDs

        # Initialize results (computed later)
        self.outcome = None
        self.overall_happiness = None
        self.strategic_voting_options = {}

    def calculate_initial_outcome(self):
        """
        Calculates the initial outcome and happiness levels based on true preferences.
        """

        self.outcome, scores = self.voting_scheme.execute_voting(self.preferences)
        self.overall_happiness = self.voting_scheme.calc_overall_happiness(self.outcome)

    def analyze_strategic_voting(self):
        """
        Analyzes strategic voting possibilities for each agent.

        * Simulates different potential voting strategies.
        * Explores collusion potential, where applicable.
        * Stores potential strategic options.
        """

        for agent_id in self.agents:
            # Analyze individual strategic options
            self.strategic_voting_options[agent_id] = self._analyze_individual_strategy(agent_id)

    def _analyze_individual_strategy(self, agent_id):
        """
        Analyzes individual strategic voting options for a single agent.

        Args:
            agent_id: The ID of the agent for which to analyze strategies.

        Returns:
            A list of dictionaries, each representing a potential strategic vote, containing:
                * 'strategic_vote': The manipulated preference list
                * 'happiness_gain': Improvement in happiness compared to truthful voting
        """

        true_prefs = self.preferences[agent_id]
        strategic_options = []

        # Generate all possible permutations of preferences
        for strategic_vote in itertools.permutations(true_prefs):
            # Simulate outcome with strategic vote
            simulated_preferences = self.preferences.copy()  # Avoid modifying original
            simulated_preferences[agent_id] = list(strategic_vote)

            simulated_outcome, scores = self.voting_scheme.execute_voting(simulated_preferences)
            simulated_happiness = self.voting_scheme.calc_ind_happiness(agent_id, simulated_outcome)

            # Calculate happiness gain (if any)
            true_happiness = self.voting_scheme.calc_ind_happiness(agent_id, self.outcome)
            happiness_gain = simulated_happiness - true_happiness

            if happiness_gain > 0:  # Strategy is beneficial
                strategic_options.append({
                    'strategic_vote': strategic_vote,
                    'happiness_gain': happiness_gain
                })

        return strategic_options

    def _analyze_joint_strategic_votes(self, coalition):
        """
        Analyzes strategic voting possibilities for a given coalition of agents.
        """

        all_joint_permutations = itertools.product(*[itertools.permutations(self.preferences[agent_id]) for agent_id in coalition])
        collusion_data = []  # Store potential collusions

        for joint_strategic_votes in all_joint_permutations:
            simulated_prefs = self.preferences.copy()
            for agent_id, strategic_vote in zip(coalition, joint_strategic_votes):
                simulated_prefs[agent_id] = list(strategic_vote)

            simulated_outcome, _ = self.voting_scheme.execute_voting(simulated_prefs)
            happiness_increases = []

            all_gains_positive = True
            for agent_id, strategic_vote in zip(coalition, joint_strategic_votes):
                simulated_happiness = self.voting_scheme.calc_ind_happiness(agent_id, simulated_outcome)
                true_happiness = self.voting_scheme.calc_ind_happiness(agent_id, self.outcome)
                happiness_increase = simulated_happiness - true_happiness
                happiness_increases.append(happiness_increase)

                if happiness_increase <= 0:
                    all_gains_positive = False
                    break

            if all_gains_positive:
                overall_happiness_increase = sum(happiness_increases) / len(coalition)
                collusion_data.append({
                    'coalition': coalition,
                    'strategic_votes': joint_strategic_votes,
                    'overall_happiness_increase': overall_happiness_increase,
                    'individual_happiness_increases': happiness_increases
                })

        # Create and return the DataFrame
        return pd.DataFrame(collusion_data)

    def _analyze_collusion(self):
        """Analyzes potential collusion strategies."""
        collusions_df = pd.DataFrame()  # Store results from all coalitions

        for coalition_size in range(2, len(self.agents) + 1):
            for coalition in itertools.combinations(self.agents, coalition_size):
                coalition_results = self._analyze_joint_strategic_votes(coalition)
                if not coalition_results.empty:
                    collusions_df = pd.concat([collusions_df, coalition_results], ignore_index=True)

        return collusions_df


    def calculate_risk(self):
        """
        Calculates a risk metric for strategic voting, taking into account
        individual and collusive strategies.
        """
        # TODO: Replace placeholder
        return random.random()  # Placeholder risk value

In [56]:
# Set up the MultiAgentTVAModel
borda_model = MultiAgentTVAModel(preferences, borda_scheme)

# Initial outcome and happiness (could be pre-calculated if you wish)
borda_model.calculate_initial_outcome()

# Analyze strategic voting
borda_model.analyze_strategic_voting()

# Example Output (Focusing on agents 'A' and 'B')
print("STRATEGIC VOTING ANALYSIS (Borda Count)")
print("---------------------------------------")

for agent_id, strategies in borda_model.strategic_voting_options.items():
    if agent_id in borda_model.agents:
        print(f"\nAgent: {agent_id}")
        print(f"True Preferences: {borda_model.preferences[agent_id]}")

        if strategies:
            print("Potential Strategic Votes:")
            for strategy in strategies:
                print(f"  - Strategic Vote: {strategy['strategic_vote']}")
                print(f"    Happiness Gain: {strategy['happiness_gain']}")
        else:
            print("No beneficial strategic votes found.")

STRATEGIC VOTING ANALYSIS (Borda Count)
---------------------------------------

Agent: A
True Preferences: ['o_1', 'o_3', 'o_2', 'o_4']
No beneficial strategic votes found.

Agent: B
True Preferences: ['o_2', 'o_1', 'o_3', 'o_4']
Potential Strategic Votes:
  - Strategic Vote: ('o_2', 'o_3', 'o_1', 'o_4')
    Happiness Gain: 0.19999999999999998
  - Strategic Vote: ('o_2', 'o_3', 'o_4', 'o_1')
    Happiness Gain: 0.19999999999999998
  - Strategic Vote: ('o_1', 'o_2', 'o_3', 'o_4')
    Happiness Gain: 0.19999999999999998
  - Strategic Vote: ('o_1', 'o_3', 'o_2', 'o_4')
    Happiness Gain: 0.6
  - Strategic Vote: ('o_1', 'o_3', 'o_4', 'o_2')
    Happiness Gain: 0.6
  - Strategic Vote: ('o_1', 'o_4', 'o_2', 'o_3')
    Happiness Gain: 0.6
  - Strategic Vote: ('o_1', 'o_4', 'o_3', 'o_2')
    Happiness Gain: 0.6
  - Strategic Vote: ('o_3', 'o_2', 'o_1', 'o_4')
    Happiness Gain: 0.19999999999999998
  - Strategic Vote: ('o_3', 'o_2', 'o_4', 'o_1')
    Happiness Gain: 0.19999999999999998
  - S

In [ ]:
df = borda_model._analyze_collusion()
print(df)

<ipython-input-55-fb9733db185e>:134: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collusions_df = collusions_df.append(coalition_results, ignore_index=True)
<ipython-input-55-fb9733db185e>:134: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collusions_df = collusions_df.append(coalition_results, ignore_index=True)
<ipython-input-55-fb9733db185e>:134: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collusions_df = collusions_df.append(coalition_results, ignore_index=True)
<ipython-input-55-fb9733db185e>:134: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  collusions_df = collusions_df.append(coalition_results, ignore_index=True)
<ipython-inp